In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes
from torchvision.ops import box_convert
from torch.utils.data import TensorDataset


from project_functions import *
from project_objects import *
from project_models import *
from project_constants import DEVICE as device
from project_constants import SEED

In [ ]:
torch.manual_seed(SEED)
torch.set_default_dtype(torch.double)

## 2 Object Localization
#### First we load and inspect the localization datasets

In [ ]:
loc_train = torch.load('data/localization_train.pt')
loc_val = torch.load('data/localization_val.pt')
loc_test = torch.load('data/localization_test.pt')

In [ ]:
print(f'Train data size: {len(loc_train)}')
print(f'Val data size: {len(loc_val)}')
print(f'Test data size: {len(loc_test)}')

In [ ]:
first_img, first_label = loc_train[0]

print(f'Shape of first image: {first_img.shape}')
print(f'Type of first image: {type(first_img)}')

print(f'\nShape of first label: {first_label.shape}')
print(f'Type of first label: {type(first_label)})')
first_label

In [ ]:
count_instances(loc_train, 'Training Data')
count_instances(loc_val, 'Validation Data')
count_instances(loc_test, 'Test Data')

#### Plotting one image from each class

In [ ]:
plot_localization_data(loc_train)

In [ ]:
plot_localization_data(loc_train, class_label=3, start_idx=10)

#### Defining a normalizer and a preprocessor

In [ ]:
loc_train_norm, loc_val_norm, loc_test_norm = normalizer(loc_train, loc_val, loc_test)

#### Defining models

In [ ]:
n_epochs = 20
batch_size = 64
lr = 0.01
momentum = 0.9
decay= 0.001

In [ ]:
train_loader = torch.utils.data.DataLoader(loc_train_norm, batch_size=batch_size, shuffle=False)
val_loader = torch.utils.data.DataLoader(loc_val_norm, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(loc_test_norm, batch_size=batch_size, shuffle=False)

loss_fn = LocalizationLoss()

In [ ]:
model_name = 'model1'

torch.manual_seed(SEED)
model1 = LocCNN1((48,60,1))
model1.to(device=device, dtype=torch.double)
optimizer = optim.SGD(model1.parameters(), lr=lr, momentum=momentum, weight_decay=decay)

loss_train1, loss_val1, train_perform1, val_perform1, losses_separated1 = train(
    n_epochs=n_epochs,
    optimizer=optimizer,
    model=model1,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    performance_calculator=localization_performance
)

plot_loss(loss_train1, loss_val1, model_name, save_model=False)
plot_lists(losses_separated1, ['detection loss', 'localization loss', 'classification loss'], model_name, save_model=False)
y_true, y_pred = predict(model1, val_loader)

In [ ]:
model_name = 'model2'

torch.manual_seed(SEED)
model2 = LocCNN2((48,60,1))
model2.to(device=device)
optimizer = optim.SGD(model2.parameters(), lr=lr, momentum=momentum, weight_decay=decay)

loss_train2, loss_val2, train_perform2, val_perform2, losses_separated2 = train(
    n_epochs=n_epochs,
    optimizer=optimizer,
    model=model2,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    performance_calculator=localization_performance
)

plot_loss(loss_train2, loss_val2, model_name, save_model=False)
plot_lists(losses_separated2, ['detection loss', 'localization loss', 'classification loss'], model_name, save_model=False)
y_true, y_pred = predict(model2, val_loader)

In [ ]:
model_name = 'model3'

torch.manual_seed(SEED)
model3 = LocCNN3((48,60,1))
model3.to(device=device)
optimizer = optim.SGD(model3.parameters(), lr=lr, momentum=momentum, weight_decay=decay)

loss_train3, loss_val3, train_perform3, val_perform3, losses_separated3 = train(
    n_epochs=n_epochs,
    optimizer=optimizer,
    model=model3,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    performance_calculator=localization_performance
)

plot_loss(loss_train3, loss_val3, model_name, save_model=False)
plot_lists(losses_separated3, ['detection loss', 'localization loss', 'classification loss'], model_name, save_model=False)
y_true, y_pred = predict(model3, val_loader)

In [ ]:
# model_name = 'model4'

# torch.manual_seed(SEED)
# model4 = LocCNN4((48,60,1))
# model4.to(device=device)
# optimizer = optim.SGD(model4.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)

# loss_train4, loss_val4, train_perform4, val_perform4, losses_separated4 = train(
#     n_epochs=n_epochs,
#     optimizer=optimizer,
#     model=model4,
#     loss_fn=loss_fn,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     performance_calculator=localization_performance
# )

# plot_loss(loss_train4, loss_val4, model_name, save_model=True)
# plot_lists(losses_separated4, ['detection loss', 'localization loss', 'classification loss'], model_name, save_model=True)
# y_true, y_pred = predict(model4, val_loader)

In [ ]:
model_name = 'model5'

torch.manual_seed(SEED)
model5 = LocCNN5((48,60,1))
model5.to(device=device)
optimizer = optim.SGD(model5.parameters(), lr=lr, momentum=momentum, weight_decay=decay)

loss_train5, loss_val5, train_perform5, val_perform5, losses_separated5 = train(
    n_epochs=n_epochs,
    optimizer=optimizer,
    model=model5,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    performance_calculator=localization_performance
)

plot_loss(loss_train5, loss_val5, model_name, save_model=False)
plot_lists(losses_separated5, ['detection loss', 'localization loss', 'classification loss'], model_name, save_model=False)
y_true, y_pred = predict(model5, val_loader)

#### Model Selection

In [ ]:
best_model, best_performance = model_selector([model1, model2, model3, model5], [val_perform1[-1],val_perform2[-1],val_perform3[-1],val_perform5[-1]])

# Print additional details of the best model
print("Best Model Details\n--------------------------------------------------------------")
print(f"Network architecture/ layout: {best_model}\n")
#print(f"Optimizer Parameters: {best_data.optimizer}")
print(f"Validation Performance: {best_performance}")

#### Model Evaluation

In [ ]:
test_performance = localization_performance(best_model, test_loader)
print(10*'-'+'Test Performance' + 10*'-')
print(f"Test Accuracy: {test_performance[0]}\nTest IOU: {test_performance[1]}\nOverall Performance: {test_performance[2]}")

In [ ]:
y_true, y_pred = predict(best_model, test_loader)
plot_predictions(loc_test, y_true, y_pred, label=3, start_idx=0)

# 3 Object Detection

#### Loading the data and inspecting the data

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes
from torchvision.ops import box_convert
from torch.utils.data import TensorDataset


from project_functions import *
from project_objects import *
from project_models import *
from project_constants import DEVICE as device
from project_constants import SEED

torch.set_default_dtype(torch.double)

In [ ]:
train_labels = torch.load('data/list_y_true_train.pt')
val_labels = torch.load('data/list_y_true_val.pt')
test_labels = torch.load('data/list_y_true_test.pt')

In [ ]:
print(f'Train label size: {len(train_labels)}')
print(f'Val label size: {len(val_labels)}')
print(f'Test label size: {len(test_labels)}')

In [ ]:
train_imgs = torch.load('data/detection_train.pt')
val_imgs = torch.load('data/detection_val.pt')
test_imgs = torch.load('data/detection_test.pt')

In [ ]:
train_labels_local = prepare_labels(train_labels, (2,3,6))
val_labels_local = prepare_labels(val_labels, (2,3,6))
test_labels_local = prepare_labels(test_labels, (2,3,6))

In [ ]:
det_train = merge_datasets(train_imgs, train_labels_local)
det_val = merge_datasets(val_imgs, val_labels_local)
det_test = merge_datasets(test_imgs, test_labels_local)

#### Plotting some images from the detection dataset

In [ ]:
imgs = [img for img,_ in train_imgs]
plot_detection_data(imgs, train_labels, start_idx=10)

#### Normalizing the dataset TBD

In [ ]:
det_train_norm, det_val_norm, det_test_norm = normalizer(det_train, det_val, det_test)

#### Defining models

In [ ]:
n_epochs = 8
batch_size = 64
lr = 0.01
momentum = 0.9
decay= 0.001

In [ ]:
train_loader = torch.utils.data.DataLoader(det_train_norm, batch_size=batch_size, shuffle=False)
val_loader = torch.utils.data.DataLoader(det_val_norm, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(det_test_norm, batch_size=batch_size, shuffle=False)

loss_fn = DetectionLoss()

In [ ]:
model_name = 'DetCNN1'
torch.manual_seed(SEED)
modelcnn1 = DetCNN1()
modelcnn1.to(device=device)
optimizer = optim.SGD(modelcnn1.parameters(), lr=lr, momentum=momentum, weight_decay=decay)

loss_train1, loss_val1, train_performance1, val_performance1, losses_separated1 = train(
    n_epochs=n_epochs,
    optimizer=optimizer,
    model=modelcnn1,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    performance_calculator=detection_performance
)

plot_loss(loss_train1, loss_val1, model_name, save_model=False)
plot_lists(losses_separated1, ['detection loss', 'localization loss', 'classification loss'], model_name, save_model=False)

In [ ]:
model_name = 'DetCNN2'
torch.manual_seed(SEED)
modelcnn2 = DetCNN2()
modelcnn2.to(device=device)
optimizer = optim.SGD(modelcnn2.parameters(), lr=lr, momentum=momentum, weight_decay=decay)

loss_train2, loss_val2, train_performance2, val_performance2, losses_separated2 = train(
    n_epochs=n_epochs,
    optimizer=optimizer,
    model=modelcnn2,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    performance_calculator=detection_performance
)

plot_loss(loss_train2, loss_val2, model_name, save_model=False)
plot_lists(losses_separated2, ['detection loss', 'localization loss', 'classification loss'], model_name, save_model=False)

In [ ]:
y_true, y_pred = predict(model, val_loader, binary_class=True)
imgs = [img for img,_ in val_imgs]
y_pred_reshaped = y_pred.permute(0,2,3,1)
y_pred_global = local_to_global_list(y_pred_reshaped)
plot_detection_data(imgs, val_labels, y_pred_global, start_idx=10)

#### Model Selection

In [ ]:
best_model, best_performance = model_selector([model], [val_performance[-1]])

# Print additional details of the best model
print("Best Model Details\n--------------------------------------------------------------")
print(f"Network architecture/ layout: {best_model}\n")
#print(f"Optimizer Parameters: {best_data.optimizer")
print(f"Validation Performance: {best_performance}")
#print(f"Validation Accuracy {round(best_data['model_man_val_accuracy'], 2)}")

#### Model Evaluation

In [ ]:


test_performance = detection_performance(best_model, test_loader)
print(10*'-'+'Test Performance' + 10*'-')
print(f"Average Precision: {test_performance[-1]}")

In [ ]:
y_true, y_pred = predict(best_model, test_loader)
plot_predictions(loc_test, y_true, y_pred, label=3, start_idx=0)